In [2]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5ForConditionalGeneration

from transformers import AutoModel, AutoConfig

from scipy.special import softmax
import numpy as np

# from peft import PeftModel


/media/trevor/SecondaryM2/VirginiaTech/HokieGuideliner/hokieguideliner/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## User Input / Memory Extraction 1

Models

In [3]:
inst_tokenizer = AutoTokenizer.from_pretrained("prakharz/DIAL-BART0")
inst_model = AutoModelForSeq2SeqLM.from_pretrained("prakharz/DIAL-BART0")

# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

sent_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
sent_model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
config = AutoConfig.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
user_input = "I like country music."

In [5]:
instruct_input = "Instruction:What is the topic of conversation?\n\nInput:[CONTEXT]{}[ENDOFDIALOGUE][QUESTION]The topic of conversation is".format(user_input)
tokens_input = inst_tokenizer(instruct_input, max_length=250, padding='max_length', truncation=True, return_tensors='pt')
input_out = inst_model.generate(**tokens_input)
topic = inst_tokenizer.decode(input_out[0], skip_special_tokens=True)
print(topic)

/media/trevor/SecondaryM2/VirginiaTech/HokieGuideliner/hokieguideliner/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Country music


In [6]:
tokens_input = sent_tokenizer(user_input, max_length=250, padding='max_length', truncation=True, return_tensors='pt')
input_out = sent_model(**tokens_input)

scores = softmax(input_out[0][0].detach().numpy())
print(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

[0.00443188 0.08218222 0.91338587]
1) positive 0.9134
2) neutral 0.0822
3) negative 0.0044


In [7]:
def extract_topic_sentiment(text_in):
    instruct_input = "Instruction:What is the topic of conversation?\n\nInput:[CONTEXT]{}[ENDOFDIALOGUE][QUESTION]The topic of conversation is".format(text_in)
    tokens_input = inst_tokenizer(instruct_input, max_length=250, padding='max_length', truncation=True, return_tensors='pt')
    input_out = inst_model.generate(**tokens_input)
    topic = inst_tokenizer.decode(input_out[0], skip_special_tokens=True)

    tokens_input = sent_tokenizer(text_in, max_length=250, padding='max_length', truncation=True, return_tensors='pt')
    input_out = sent_model(**tokens_input)

    scores = softmax(input_out[0][0].detach().numpy())
    #print(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")

    return topic, config.id2label[ranking[0]]

In [83]:
# THIS CELL IS JUST USED FOR DATASET MANIPULATION
# file = open('./CoT/generated_ds.csv', 'r')
# f2 = open('./CoT/ds.txt', 'w')
# lines = file.readlines()

# for line in lines:
#     idx = line.find(':')
#     l2 = '[' + line[:idx]  + ']|' + line[idx+1:]
#     f2.write(l2)

AI TOPIC GENERATOR

In [8]:
inst_tokenizer2 = AutoTokenizer.from_pretrained("prakharz/DIAL-BART0")
inst_model2 = AutoModelForSeq2SeqLM.from_pretrained("prakharz/DIAL-BART0")

inst_model2.load_state_dict(torch.load('./model/topic_er.pt'))

<All keys matched successfully>

In [9]:
# chain of topics
def generate_cot(text_in):
    tok_text = inst_tokenizer2(text_in, return_tensors='pt')
    gen_text = inst_model2.generate(**tok_text)
    dec_text = inst_tokenizer2.decode(gen_text[0], skip_special_tokens=True)
    return dec_text

In [18]:
print(extract_topic_sentiment("I like Abraham Lincoln"))
print(generate_cot("Abraham Lincoln"))

/media/trevor/SecondaryM2/VirginiaTech/HokieGuideliner/hokieguideliner/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


1) positive 0.7394
2) neutral 0.2476
3) negative 0.013
('Abraham Lincoln', 'positive')
[History, American Revolution, Presidents/Prime Ministers]


## Memory Update

In [10]:
# META-TOPIC : {Music, Sports, Games}
# SUB-TOPIC : {Rock n Roll, Country, etc.}
# MICRO-TOPIC : {Journey, Boston, ACDC, Aerosmith}
memory = {}

def add_to_memory(mem, memory, idx, key=None, value=None):
    # print(mem)
    if len(mem) == 0:
        #print(memory.keys())
        if key != None:
            if key in memory.keys():
                memory[key] = value
            else:
                memory[key] = value

        return memory
    
    if mem[0] not in memory.keys():
        memory[mem[0]] = {}
        
        if key != None:
            #print(mem[0])
            #print(memory[mem[0]].keys())
            if key not in memory[mem[0]].keys():
                memory[mem[0]] = {key:value}

    add_to_memory(mem[1:], memory[mem[0]], idx+1, key, value)
    return memory

print(add_to_memory(['Animals', 'Marine Life', 'Sharks', 'Beach'], memory, 0))
print(add_to_memory(['Animals', 'Marine Life', 'Sharks', 'Sand'], memory, 0, 'relationship', 'dislike'))
print()
print(add_to_memory(['Animals', 'Marine Life', 'Sharks', 'Sand'], memory, 0, 'relationship', 'like'))
print(add_to_memory(['Animals', 'Marine Life', 'Sharks', 'Sand'], memory, 0, 'strength', 5))

# print(memory['Animals']['Marine Life']['Sharks']['Sand'].keys())

{'Animals': {'Marine Life': {'Sharks': {'Beach': {}}}}}
{'Animals': {'Marine Life': {'Sharks': {'Beach': {}, 'Sand': {'relationship': 'dislike'}}}}}

{'Animals': {'Marine Life': {'Sharks': {'Beach': {}, 'Sand': {'relationship': 'like'}}}}}
{'Animals': {'Marine Life': {'Sharks': {'Beach': {}, 'Sand': {'relationship': 'like', 'strength': 5}}}}}


## Memory Retreive

In [11]:
def read_memory(mem, memory, idx):
    if len(mem) == 0:
        return memory
    return read_memory(mem[1:], memory[mem[0]], idx+1)

print(read_memory(['Animals', 'Marine Life', 'Sharks'], memory, 0))

{'Beach': {}, 'Sand': {'relationship': 'like', 'strength': 5}}


## Node2Profile

In [12]:
print(str(read_memory(['Animals', 'Marine Life', 'Sharks'], memory, 0))+" <- IS A STRING")

{'Beach': {}, 'Sand': {'relationship': 'like', 'strength': 5}} <- IS A STRING


## Guideline Generate

In [ ]:
# THE SAME

## Response Generate

In [ ]:
# THE SAME

## Memory Extraction 2

In [15]:
# USE METHOD FROM MEMORY EXTRACTION 1
ext = extract_topic_sentiment("I do not like reading non-fiction books.")
# WILL NEED TO RUN THIS SENTENCE BY SENTENCE?
print(ext)

cot = generate_cot(ext[0])
print(cot)

1) negative 0.9059
2) neutral 0.0821
3) positive 0.012
('Non-fiction', 'negative')
[Literature, Science Fiction, Novels]


## Self-checking